📌**Project Overview**

This project builds a **Named Entity Recognition (NER)** model using **BERT**.

 🎯 **Goal**

The goal is to train a model that can find and classify entities in text.
For example:

* Person name
* Organization
* Skills
* Location
* `Or any custom labels in the dataset`

---

 🧠**What the code does**

1. **Load Dataset**

   * Load a prepared NER dataset from disk.
   * Dataset contains tokens (words) and their labels.

2. **Prepare Labels**

   * Convert label names to numbers.
   * Create mapping between label and id.

3. **Tokenization**

   * Use BERT tokenizer.
   * Split words into subwords.
   * Align labels with new tokens.

4. **Build Model**

   * Load `bert-base-uncased`.
   * Add token classification head.
   * Set number of labels.

5. **Prepare Training**

   * Use DataCollator for correct padding.
   * Set training parameters (learning rate, batch size, epochs).

6. **Evaluation**

   * Use `seqeval` to calculate:

     * Precision
     * Recall
     * F1 Score
     * Accuracy

7. **Training**

   * Train the model on training data.
   * Evaluate on validation data.
   * Fine-tune BERT for NER task.

---

 🏁 **Final Result**

After training:

* We get a fine-tuned BERT model.
* The model can predict entity labels for each word in a sentence.
* The model can be saved and used in backend (API or application).

---

📌 **About the Author**

 - **Name:** **Mohamed Mamdouh**
 - Student at the Faculty of **Artificial Intelligence**

 - [**LinkedIn**](https://www.linkedin.com/in/ai-mohamed-mamdouh-74043b331/)
 - [**GitHub**](https://github.com/ai-mohamed-mamdouh)
 - [**Kaggle**](https://www.kaggle.com/mohamed00mamdouh)

In [ ]:
!pip install --upgrade transformers
!pip install tqdm

# **Imports & Load DataSet**

In [ ]:
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer
)
import numpy as np

# Goal: We import tools for:
# - loading dataset (datasets)
# - tokenizer + model + training (transformers)
# - math (numpy)


# =========================
# Load dataset from disk
# =========================
from datasets import load_from_disk
data = load_from_disk("YOUR DATA PATH")
print(data)

# Goal: Load the NER dataset saved before.
# Why: We want to train a model on this dataset.
# Output: A DatasetDict like: {"train", "validation", "test"} (maybe)

# **Build label dictionaries**

In [ ]:
# Goal: Collect all label IDs used in the dataset.
# Example: labels can be 0, 1, 2... or like "B-SKILL", "I-SKILL", "O" (depends on dataset).
all_labels = sorted(list({tag for sent in data['train']["ner_tags"] for tag in sent}))

# Goal: Create mapping label -> id
label2id = {l: i for i, l in enumerate(all_labels)}

# Goal: Create mapping id -> label
id2label = {i: l for l, i in label2id.items()}

# Why we need this:
# - The model works with numbers (ids).
# - For metrics and display we want label names too.

# **Convert ner_tags to labels (ids)**

In [ ]:
def encode_labels(example):
    # Goal: Add a new key "labels" to each example.
    # Why: Trainer expects the label column to be named "labels".
    example["labels"] = [label2id[t] for t in example["ner_tags"]]
    return example

data = data.map(encode_labels)

# Output now:
# each example has:
# - tokens (list of words)
# - ner_tags (original labels)
# - labels (numeric labels)

# **Tokenize and align labels**

In [ ]:
# Load tokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Goal: Use BERT tokenizer.
# Why: BERT splits words into subwords. Example: "playing" -> "play", "##ing".
# This creates a problem: labels are for words, but tokens become subwords.
# So we must align labels to tokens (next step).


# =========================
# Tokenize and align labels
# =========================
def tokenize_and_align_labels(examples):
    # Goal: Tokenize the input tokens (already split words).
    tokenized = tokenizer(
        examples["tokens"],
        is_split_into_words=True,  # Input is list of words (not a full string)
        truncation=True,           # Cut if too long
        padding=False              # No padding here (collator will do it later)
    )

    aligned_labels = []

    # Goal: For each sentence in the batch, align labels to tokenized output.
    for i in range(len(examples["tokens"])):
        # word_ids tells for each token which original word it came from.
        # Example: [None, 0, 1, 1, 2, None]
        # None = special token like [CLS] [SEP]
        word_ids = tokenized.word_ids(batch_index=i)

        prev_word_id = None
        label_ids = []

        for word_id in word_ids:
            if word_id is None:
                # Special tokens have no label.
                # We use -100 so loss will ignore it.
                label_ids.append(-100)

            elif word_id != prev_word_id:
                # First sub-token of a word:
                # take the real label for that word.
                label_ids.append(examples["labels"][i][word_id])

            else:
                # Other sub-tokens of the same word:
                # ignore them in loss to avoid double-counting.
                label_ids.append(-100)

            prev_word_id = word_id

        aligned_labels.append(label_ids)

    # Goal: Put aligned labels inside tokenized output.
    tokenized["labels"] = aligned_labels
    return tokenized

data = data.map(tokenize_and_align_labels, batched=True)

# Output:
# dataset now has fields like:
# - input_ids
# - attention_mask
# - labels (aligned with input_ids length)

# **Load model for Token Classification (NER)**

In [ ]:
ner_model_name = 'bert-base-NER'  # Note: This variable is not used later.

model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(all_labels),  # number of classes
    id2label=id2label,           # id -> label name
    label2id=label2id            # label name -> id
)

# Goal: Create a BERT model with a classification head for each token.
# Why: NER predicts a label for every token (word/subword).

# **Data collator for token classification**

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

# Goal: Make batches with correct padding.
# Why:
# - Each sentence has different length.
# - This collator pads input_ids and attention_mask.
# - It also pads labels with -100 so loss ignores padding tokens.

# **Training**

In [ ]:
# =========================
# Training arguments
# =========================
args = TrainingArguments(
    output_dir="./ner_bert_ar",          # where to save outputs
    learning_rate=2e-5,                 # small LR for BERT fine-tuning
    per_device_train_batch_size=32,     # train batch size
    num_train_epochs=1,                 # number of epochs
    weight_decay=0.01,                  # regularization
    logging_steps=10,                   # log every 10 steps
    save_strategy="no",                 # do not save checkpoints
    report_to="none",                   # no wandb / tensorboard reports

    load_best_model_at_end=True,        # keep best model after training (needs evaluation + saving)
    metric_for_best_model="f1",         # best model = highest f1
    per_device_eval_batch_size=32,      # eval batch size
    # evaluation_strategy="epoch"       # IMPORTANT: if commented, evaluation may NOT run automatically
)

# Important note:
# - load_best_model_at_end=True usually needs:
#   evaluation_strategy + save_strategy not "no"
#   (otherwise Trainer may not know "best model" because it never saves checkpoints)

In [ ]:
# =========================
# Metrics (F1, precision, recall...) using seqeval
# =========================
import evaluate
import numpy as np

seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    # p = (predictions, labels)
    predictions, labels = p

    # Goal: Convert model outputs (logits) to predicted class id.
    predictions = np.argmax(predictions, axis=2)

    true_predictions = []
    true_labels = []

    # Goal: Remove tokens with label -100 (ignored tokens).
    for pred, lab in zip(predictions, labels):
        temp_preds = []
        temp_labels = []

        for p_i, l_i in zip(pred, lab):
            if l_i != -100:
                # Convert ids to label names (needed by seqeval)
                temp_preds.append(id2label[p_i])
                temp_labels.append(id2label[l_i])

        true_predictions.append(temp_preds)
        true_labels.append(temp_labels)

    # Goal: Compute seqeval metrics for NER.
    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    # Return main metrics to Trainer
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
# =========================
# Trainer
# =========================
trainer = Trainer(
    model=model,                         # the NER model
    args=args,                           # training settings
    train_dataset=data["train"],         # training split
    eval_dataset=data["validation"],     # validation split (important for F1)
    # tokenizer=tokenizer,               # optional in new versions; can be used for saving / padding info
    data_collator=data_collator,         # padding + label padding
    compute_metrics=compute_metrics       # calculate F1 etc.
)

# Goal: Trainer controls training loop:
# - batching
# - forward/backward
# - evaluation
# - logging

In [ ]:
# =========================
# Train the model
# =========================

trainer.train()

# Goal: Fine-tune BERT on your NER dataset.
# Output: Trained model weights (in memory, and maybe in output_dir depending on settings)

In [ ]:
# =========================
# Train the model
# =========================

test_results = trainer.evaluate(data["test"])

print(test_results)

In [ ]:
# =========================
# Save the model
# =========================

# Define folder name
save_path = "./my_ner_model"

# Save model
trainer.save_model(save_path)

# Save tokenizer
tokenizer.save_pretrained(save_path)

# A folder will be created:
'''
my_ner_model/
    config.json
    pytorch_model.bin
    tokenizer.json
    vocab.txt
    special_tokens_map.json
    tokenizer_config.json
'''

# **Inference**

In [ ]:
#  Load pipeline
from transformers import pipeline

# Goal: Create NER pipeline (token-classification)
ner_pipe = pipeline(
    task="token-classification",
    model="./my_ner_model",          # folder path (or use your model object)
    tokenizer="./my_ner_model",
    aggregation_strategy="simple"    # merge sub-tokens into full words
)

In [ ]:
# predict

text = "Ahmed is skilled in Python, Java, and Machine Learning."
preds = ner_pipe(text)

#  Print results
for p in preds:
    print(p)

# O,O,O,O, B-SKILL , O ,  B-SKILL , O , O , B-SKILL , O , I-SKILL , O


---
**✅ Project Summary**

In this project, we trained a **BERT model for Named Entity Recognition (NER)**.

We loaded a prepared dataset that contains:

* Tokens (words)
* NER labels for each word

Then we:

* Converted labels to numeric IDs
* Tokenized the text using BERT tokenizer
* Aligned labels with subword tokens
* Built a token classification model using `bert-base-uncased`
* Used HuggingFace `Trainer` for training
* Evaluated the model using Precision, Recall, F1-score, and Accuracy

After training, the model can:

* Predict entity labels for each word in a sentence
* Be saved and used in a backend system
* Be used with pipeline for inference

This notebook shows the full workflow:
Dataset → Preprocessing → Training → Evaluation → Ready Model 🚀
